In [2]:
%reset -f

import numpy as np
import copy
import time

from qiskit.opflow.primitive_ops import PauliSumOp
from qiskit.quantum_info import SparsePauliOp

from qiskit.primitives import Estimator as PrimitiveEstimator
from qiskit import QuantumCircuit, Aer
from qiskit.circuit import ParameterVector

import networkx as nx
import scipy

# List of optimizers: https://qiskit.org/documentation/stable/0.28/apidoc/qiskit.algorithms.optimizers.html     

print("Cell Evaluated")


Cell Evaluated


# Scale factor optimization





The aim of the scale factor is to characterize how noise in the device will affect the final energy obtained through VQE. The scale factor is a number which allows us to rescale the final expectation value after optimization to a value that is closer to the true value. For an operator $\hat{M}$, the scale factor is given as follows:

\begin{align*}
\mathcal{f} = \frac{\langle \hat{M} \rangle _{\textrm{noiseless}}}{\langle \hat{M} \rangle _{\textrm{hardware}}}
\end{align*}

which allows us to scale our measured ground state energy estimate according to:

\begin{align*}
E_g = \mathcal{f} \langle 0 | U^\dagger\left( \theta \right) \hat{H} U\left( \theta \right) | 0 \rangle
\end{align*}

In our case, we were given the ground state energy of $\hat{H}_{\textrm{kagome}}$. The ideal scale factor can be obtained using $\; \mathcal{f} = \frac{ \langle \hat{H}_{\textrm{kagome}}  \rangle _{\textrm{noiseless}}  }{  \langle \hat{H}_{\textrm{kagome}}  \rangle _{\textrm{hardware}} }$. In general however, the ground state energy of the Hamiltonian we are interested in is not known *a priori*. In addition, calculating $\langle \hat{M} \rangle _{\textrm{noiseless}}$ might not be straightforward especially for larger system sizes and complicated ansatzes. Hence, for our approach to be feasible, our choice of $\hat{M}$ is important.

To do this, we introduce a new Hamiltonian: 

\begin{align*}
\hat{H}_t = X_1 X_2 + Y_1 Y_2 + Z_1 Z_2
\end{align*}


which only acts on 2 qubits but with the same interaction as that in $\hat{H}_{\textrm{kagome}}$. Being only a 2-qubit Hamiltonian, this $4\times4$ matrix is easily diagonalizable with a ground state that can be easily prepared and therefore be used to approximate $\mathcal{f}$.

$\hat{H}_t$ has the ground state energy of $-3$ with the ground state $\frac{1}{\sqrt{2}}(|01\rangle - |01\rangle)$.

To further improve the accuracy of our scale factor:

- Apply $\hat{H}_t$ to all qubits `ibmq_guadalupe` that we intend to use, paired according to their connectivity in the topology graph. 

  The qubits that we have chosen to use for VQE in `ibmq_guadalupe` consists of the inner ring of 12 qubits (as per the demo), which we decomposed into 2 perfect matchings $\mathcal{G}_1$ and $\mathcal{G}_2$, each consisting of 6 pairs of qubits. $\hat{H}_t$ can be applied to each pair of qubits, yielding the 12-qubit Hamiltonians: 
  
  \begin{align*}
  \hat{\mathcal{G}}_1 = \sum_{\langle i, j \rangle \in \mathcal{G}_1} \left( X_i X_j + Y_i Y_j + Z_i Z_j \right) 
  \text{ and } 
  \hat{\mathcal{G}}_2 = \sum_{\langle i, j \rangle \in \mathcal{G}_2} \left( X_i X_j + Y_i Y_j + Z_i Z_j \right)
  \end{align*}

  
  Our scale factor can then be found using $\mathcal{f} = \frac{1}{2}\left( \frac{ \langle \hat{\mathcal{G}}_1 \rangle _\textrm{noiseless} }{ \langle \hat{\mathcal{G}}_1 \rangle _\textrm{hardware} } + \frac{ \langle \hat{\mathcal{G}}_2 \rangle _\textrm{noiseless} }{ \langle \hat{\mathcal{G}}_2 \rangle _\textrm{hardware} } \right)$

  Since $\hat{H}_t$ act on 6 independent pairs of qubits in $\hat{\mathcal{G}}_1$ and $\hat{\mathcal{G}}_2$, the ground state energy of $\hat{\mathcal{G}}_1$ and $\hat{\mathcal{G}}_2$ is simply $E_g^{\hat{\mathcal{G}}_1} = E_g^{\hat{\mathcal{G}}_2} = 6 \times (-3) = 18$ without the need to diagonalize a $4096\times4096$ matrix. We emphasize that the values of $E_g^{\hat{\mathcal{G}}_1}$ and $E_g^{\hat{\mathcal{G}}_2}$ is used to approximate $\mathcal{f}$, and not directly infer the ground state energy of the Kagome Hamiltonian, despite having the same values.
  
- We use the same VQE ansatz used to find the ground state of $\hat{H}_{\textrm{kagome}}$ to prepare the ground states of $\hat{\mathcal{G}}_1$ and $\hat{\mathcal{G}}_2$. The ground states for both Hamiltonians are also found using VQE. This ensures that the circuit depth and gates used are consistent.
  

The Hamiltonians $\hat{\mathcal{G}}_1$ and $\hat{\mathcal{G}}_2$ are simple enough for preparation of their ground states to be feasibly simulated, and the exact value of the ground state energy easily found. For a different problem Hamiltonian, one might need to choose a different target $\hat{M}$.



In [3]:
def cryansatz(nqubits, layers, init_bitstring): # Hardware efficient ansatz that is spin preserving.
    
    assert nqubits%2 == 0
    
    print(f"{'':+^64}\n",
      f"{' Spin Preserving CRY Ansatz (Ring Topo) ':^64}",
      f"\n{'':+^64}")
    
    my_circ = QuantumCircuit(nqubits)
    nparams = int((nqubits/2) * layers)
    
    params = ParameterVector('θ', nparams)
    params_idx = 0
    
    for x_ in range(len(init_bitstring)):
        if init_bitstring[x_] == "1":
            my_circ.x(x_)
            
    my_circ.barrier()
    
    for l_ in range(layers):
        if l_%2 ==0:
            for q2_ in range(0, nqubits,2):
                if q2_+1 == nqubits:
                    qb1 = q2_
                    qb2 = 0
                else:
                    qb1 = q2_
                    qb2 = q2_ + 1
                    
                my_circ.cx(qb1, qb2)
                my_circ.cry(params[params_idx], qb2, qb1)
                params_idx += 1
                my_circ.cx(qb1, qb2)
                my_circ.rzz(np.pi/2, qb2, qb1)
        else:
            for q2_ in range(1, nqubits,2):
                if q2_+1 == nqubits:
                    qb1 = q2_
                    qb2 = 0
                else:
                    qb1 = q2_
                    qb2 = q2_ + 1
                    
                my_circ.cx(qb1, qb2)
                my_circ.cry(params[params_idx], qb2, qb1)
                params_idx += 1
                my_circ.cx(qb1, qb2)
                my_circ.rzz(np.pi/2, qb2, qb1)

                
        my_circ.barrier()
                
    return my_circ


def gatecount(circuit):
    return len([circuit.data[gate][0].name for gate in range(len(circuit.data))])






# Last 4 qubits are not used. These unused qubits will be mapped to 0, 6, 9 15 on the device
# using initial_layout argument during the transpile stage.
drawansatz = cryansatz(12, 4, '101010101010')

print("Gate count:", gatecount(drawansatz))
print("Cicuit depth:", drawansatz.depth())

drawansatz.draw(fold=5000)



++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
              Spin Preserving CRY Ansatz (Ring Topo)              
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Gate count: 107
Cicuit depth: 17


┌───┐ ░      ┌──────────┐                ░      ┌───┐                               ┌───┐                     ░      ┌───────────┐                ░      ┌───┐                               ┌───┐                     ░ 
 q_0: ┤ X ├─░───■──┤ Ry(θ[0]) ├──■───■─────────░──────┤ X ├───────────────────■───────────┤ X ├───────────■─────────░───■──┤ Ry(θ[12]) ├──■───■─────────░──────┤ X ├───────────────────■───────────┤ X ├───────────■─────────░─
      └───┘ ░ ┌─┴─┐└────┬─────┘┌─┴─┐ │ZZ(π/2)  ░      └─┬─┘ ┌──────────┐      │           └─┬─┘           │         ░ ┌─┴─┐└─────┬─────┘┌─┴─┐ │ZZ(π/2)  ░      └─┬─┘┌───────────┐      │           └─┬─┘           │         ░ 
 q_1: ──────░─┤ X ├─────■──────┤ X ├─■─────────░───■────┼───┤ Ry(θ[6]) ├──────┼────────■────┼───■─────────┼─────────░─┤ X ├──────■──────┤ X ├─■─────────░───■────┼──┤ Ry(θ[18]) ├──────┼────────■────┼───■─────────┼─────────░─
      ┌───┐ ░ └───┘┌──────────┐└───┘           ░ ┌─┴─┐  │   └────┬─────┘      │      ┌─┴─┐  │   │ZZ(π/2)  │         ░ └───┘┌───────────┐└───┘           ░ ┌─┴─┐  │  └─────┬─────┘      │      ┌─┴─┐  │   │ZZ(π/2)  │         ░ 
 q_2: ┤ X ├─░───■──┤ Ry(θ[1]) ├──■───■─────────░─┤ X ├──┼────────■────────────┼──────┤ X ├──┼───■─────────┼─────────░───■──┤ Ry(θ[13]) ├──■───■─────────░─┤ X ├──┼────────■────────────┼──────┤ X ├──┼───■─────────┼─────────░─
      └───┘ ░ ┌─┴─┐└────┬─────┘┌─┴─┐ │ZZ(π/2)  ░ └───┘  │   ┌──────────┐      │      └───┘  │             │         ░ ┌─┴─┐└─────┬─────┘┌─┴─┐ │ZZ(π/2)  ░ └───┘  │  ┌───────────┐      │      └───┘  │             │         ░ 
 q_3: ──────░─┤ X ├─────■──────┤ X ├─■─────────░───■────┼───┤ Ry(θ[7]) ├──────┼────────■────┼───■─────────┼─────────░─┤ X ├──────■──────┤ X ├─■─────────░───■────┼──┤ Ry(θ[19]) ├──────┼────────■────┼───■─────────┼─────────░─
      ┌───┐ ░ └───┘┌──────────┐└───┘           ░ ┌─┴─┐  │   └────┬─────┘      │      ┌─┴─┐  │   │ZZ(π/2)  │         ░ └───┘┌───────────┐└───┘           ░ ┌─┴─┐  │  └─────┬─────┘      │      ┌─┴─┐  │   │ZZ(π/2)  │         ░ 
 q_4: ┤ X ├─░───■──┤ Ry(θ[2]) ├──■───■─────────░─┤ X ├──┼────────■────────────┼──────┤ X ├──┼───■─────────┼─────────░───■──┤ Ry(θ[14]) ├──■───■─────────░─┤ X ├──┼────────■────────────┼──────┤ X ├──┼───■─────────┼─────────░─
      └───┘ ░ ┌─┴─┐└────┬─────┘┌─┴─┐ │ZZ(π/2)  ░ └───┘  │   ┌──────────┐      │      └───┘  │             │         ░ ┌─┴─┐└─────┬─────┘┌─┴─┐ │ZZ(π/2)  ░ └───┘  │  ┌───────────┐      │      └───┘  │             │         ░ 
 q_5: ──────░─┤ X ├─────■──────┤ X ├─■─────────░───■────┼───┤ Ry(θ[8]) ├──────┼────────■────┼───■─────────┼─────────░─┤ X ├──────■──────┤ X ├─■─────────░───■────┼──┤ Ry(θ[20]) ├──────┼────────■────┼───■─────────┼─────────░─
      ┌───┐ ░ └───┘┌──────────┐└───┘           ░ ┌─┴─┐  │   └────┬─────┘      │      ┌─┴─┐  │   │ZZ(π/2)  │         ░ └───┘┌───────────┐└───┘           ░ ┌─┴─┐  │  └─────┬─────┘      │      ┌─┴─┐  │   │ZZ(π/2)  │         ░ 
 q_6: ┤ X ├─░───■──┤ Ry(θ[3]) ├──■───■─────────░─┤ X ├──┼────────■────────────┼──────┤ X ├──┼───■─────────┼─────────░───■──┤ Ry(θ[15]) ├──■───■─────────░─┤ X ├──┼────────■────────────┼──────┤ X ├──┼───■─────────┼─────────░─
      └───┘ ░ ┌─┴─┐└────┬─────┘┌─┴─┐ │ZZ(π/2)  ░ └───┘  │   ┌──────────┐      │      └───┘  │             │         ░ ┌─┴─┐└─────┬─────┘┌─┴─┐ │ZZ(π/2)  ░ └───┘  │  ┌───────────┐      │      └───┘  │             │         ░ 
 q_7: ──────░─┤ X ├─────■──────┤ X ├─■─────────░───■────┼───┤ Ry(θ[9]) ├──────┼────────■────┼───■─────────┼─────────░─┤ X ├──────■──────┤ X ├─■─────────░───■────┼──┤ Ry(θ[21]) ├──────┼────────■────┼───■─────────┼─────────░─
      ┌───┐ ░ └───┘┌──────────┐└───┘           ░ ┌─┴─┐  │   └────┬─────┘      │      ┌─┴─┐  │   │ZZ(π/2)  │         ░ └───┘┌───────────┐└───┘           ░ ┌─┴─┐  │  └─────┬─────┘      │      ┌─┴─┐  │   │ZZ(π/2)  │         ░ 
 q_8: ┤ X ├─░───■──┤ Ry(θ[4]) ├──■───■─────────░─┤ X ├──┼────────■────────────┼──────┤ X ├──┼───■─────────┼─────────░───■──┤ Ry(θ[16]) ├──■───■─────────░─┤ X ├──┼────────■────────────┼──────┤ X ├──┼

# Creating the rotosolve optimizer

https://arxiv.org/pdf/1905.09692.pdf

 
Other optimizers we would like to try given more time: Fraxis, RotoSelect

In [4]:
def rotosolve(parameters, circuit, hamiltonian, estimator):
    
    #https://arxiv.org/pdf/1905.09692.pdf
    
    for param_idx in range(circuit.num_parameters):
        cf = estimator.run([circuit], [hamiltonian], [parameters]).result().values[0]
        
        param_plus, param_minus = copy.deepcopy(parameters), copy.deepcopy(parameters)
        param_plus[param_idx] = parameters[param_idx] + np.pi/2
        cf_plus = estimator.run([circuit], [hamiltonian], [param_plus]).result().values[0]
        
        param_minus[param_idx] = parameters[param_idx] - np.pi/2
        cf_minus = estimator.run([circuit], [hamiltonian], [param_minus]).result().values[0]
        
        parameters[param_idx] = parameters[param_idx] - np.pi/2 - np.arctan2(2*cf - cf_plus - cf_minus, cf_plus - cf_minus)

    return parameters

print("Cell Evaluated")




Cell Evaluated


# Simple Hamiltonian

To find our scale factor, we start with a simple $2$-qubit Hamiltonian with the same $XX, YY, ZZ$ terms as the Kagome Hamiltonian:


$\hat{H} = X_1 X_2 + Y_1 Y_2 + Z_1 Z_2$

This produces a $4\times4$ matrix that is easily diagonalizable but still contains the same interaction between qubits. We find the ground state energy of this to be $-3$, with the ground state $\frac{1}{\sqrt{2}}(|01\rangle - |01\rangle)$.





In [5]:
Ham_2_obs = SparsePauliOp.from_sparse_list([("XX", [0, 1], 1), ("YY", [0, 1], 1), ("ZZ", [0,1], 1)], num_qubits=2)
Ham_2 = PauliSumOp(Ham_2_obs)
print(Ham_2.to_matrix())


print("Finding eigenvalues...")
ghl1 = time.time()
eigvals2, eigvecs2 = scipy.linalg.eigh(Ham_2.to_matrix())

assert np.all(eigvals2[:-1] <= eigvals2[1:])

print("GS energy:", np.min(eigvals2))
print("Ground state:", eigvecs2[:,0])
print("Time taken: %.6fs, cell evaluated." %(time.time() -  ghl1))

[[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j  2.+0.j  0.+0.j]
 [ 0.+0.j  2.+0.j -1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  1.+0.j]]
Finding eigenvalues...
GS energy: -2.999999999999998
Ground state: [ 0.        +0.j  0.70710678+0.j -0.70710678-0.j  0.        +0.j]
Time taken: 0.001985s, cell evaluated.


### Constructing our simple, 12-qubit Hamiltonian.

$\hat{\mathcal{G}}_1$ and $\hat{\mathcal{G}}_2$ are essentially the same Hamiltonians just acting on different qubits. This means that all we need is to construct a single Hamiltonian, $\hat{\mathcal{G}}$, and minimize $\langle \hat{\mathcal{G}} \rangle$ with our ansatz. With our optimal circuit, implementing $\hat{\mathcal{G}}_1$ and $\hat{\mathcal{G}}_2$ on `ibmq_guadalupe` can be done by changing the layout of the qubits during transpilation and choosing which qubits the interactions act on. 

This notebook just minimizes $\langle \hat{\mathcal{G}} \rangle$ without any transpilation or mapping to the qubits. This is done later on when finding the scale factor. More details are also in that notebook.

We create $\langle \hat{\mathcal{G}} \rangle$ in Python below. `edge_list` runs over separate pairs of qubits from `0` to `11`, and we will run our VQE using the 12-qubit ansatz above. 

We will then use Qiskit's transpiler options to map the qubits (and Hamiltonian interactions) to the edges in $\mathcal{G}_1$ and $\mathcal{G}_2$ to find the scale factor.


In [6]:
edge_list = [[0,1],[2,3],[4,5],[6,7],[8,9],[10,11]]

observables = [] 
for conn in edge_list:
    observables.append(("XX", conn, 1))
    observables.append(("YY", conn, 1))
    observables.append(("ZZ", conn, 1))


observables = SparsePauliOp.from_sparse_list(observables,num_qubits = 12)
ham_12 = PauliSumOp(observables)

print("Hamiltonian Created")




Hamiltonian Created


# Running the VQE using simulators


Minimizing towards the ground state of $\hat{\mathcal{G}}_1$ and $\hat{\mathcal{G}}_2$ is done fairly easily, especially due to the earlier success of the Rotosolve optimizer. 




In [9]:

# Coding the starting points (rng seeds) this way ensures that you're unlikely
# to reuse the same seed twice, yet lets you keep the start points in case you want to save them
# Setting randomstate to None shows that we are truly random, not cherry picking ideal starting points for submission
n_stpt = 10
rt1 = np.random.RandomState(None) 
seedlist = np.arange(99999) 
rt1.shuffle(seedlist)
stpt_seeds = {x:seedlist[x] for x in range(n_stpt)}

shots = None
maxiter = 100
circuit_depth = 4
init_state = '101010101010'

vqe_circ = cryansatz(12, circuit_depth, init_state)

estimator = PrimitiveEstimator([vqe_circ], [ham_12],  options={'shots':shots})

for stpt in range(n_stpt):
    intermediate_info = []
    thetlist = []

    ls2 = np.random.RandomState(stpt_seeds[stpt])
    var_params = ls2.uniform(-np.pi,np.pi, vqe_circ.num_parameters)
    init_cf = estimator.run([vqe_circ], [ham_12], [var_params]).result().values[0]

    intermediate_info.append(init_cf)
    thetlist.append(var_params)

    print("Begin optimization run. Number of parameters: %.d. Stpt seed: %.d, Max iter: %.d, " 
          %(vqe_circ.num_parameters, stpt_seeds[stpt], maxiter),
          "Init cf: %.5f" %(init_cf)
          )

    ggty1 = time.time()
    for it_ in range(maxiter):
        var_params = rotosolve(var_params, vqe_circ, ham_12, estimator)
        intermediate_cf = estimator.run([vqe_circ], [ham_12], [var_params]).result().values[0]
        intermediate_info.append(intermediate_cf)
        thetlist.append(var_params)


        if it_%5 == 0:
            print(f"Current iter: {it_:<3d}, CF calls: {3*vqe_circ.num_parameters*(it_+1):<2d}, Time elapsed: {(time.time() - ggty1):<3.1f}s, "
                  # f"Est time left: {est_time_remaining:<3.1f}s, Est total time: {est_tot_time:<3.1f}s"
                  f"Current cf val: {intermediate_cf:<6.3f}"
                  )


        # Terminates if CF somehow increases for more than a set number of iterations
        # Terminates if CF found is close to the ground state
        # Terminates if CF found does not improve
        if len(intermediate_info) > 10 and np.allclose(intermediate_info[-4:], sorted(intermediate_info[-4:])):
            break
        if np.isclose(intermediate_info[-1], -18):
            break
        if len(intermediate_info) > 10 and np.isclose(intermediate_info[-1], intermediate_info[-2]):
            break

    best_index = np.argmin(intermediate_info)
    best_cf = intermediate_info[best_index]
    best_params = thetlist[best_index]
    cf_calls = 3*vqe_circ.num_parameters*len(intermediate_info)

    print("Minimum energy found: %.5f, CF calls: %.d, stpt seed: %.d" 
          %(best_cf, cf_calls, stpt_seeds[stpt]))
    print(f'Time taken: (s): {time.time() - ggty1:.2f}')


    if best_cf <= -17.9:

        # Removes all the other iterations once the minimum has been found.
        intermediate_info = copy.deepcopy(intermediate_info[:best_index+1])
        thetlist = copy.deepcopy(thetlist[:best_index+1])

        paramdict = {"seed":stpt_seeds[stpt], "opt params": best_params, "opt cf": best_cf, "cflist": intermediate_info, "paramlist":thetlist}
        np.save("TrivH-cryansatz-12l"+str(circuit_depth)+"-notranspile-Rotosolve" +str(stpt_seeds[stpt]) + ".npy", paramdict)
        print("Saved result")
        

    print()


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
              Spin Preserving CRY Ansatz (Ring Topo)              
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Begin optimization run. Number of parameters: 24. Stpt seed: 25872, Max iter: 100,  Init cf: -3.38958
Current iter: 0  , CF calls: 72, Time elapsed: 0.6s, Current cf val: -10.829
Current iter: 5  , CF calls: 432, Time elapsed: 3.1s, Current cf val: -18.000
Minimum energy found: -17.99994, CF calls: 504, stpt seed: 25872
Time taken: (s): 3.10
Saved result

Begin optimization run. Number of parameters: 24. Stpt seed: 50905, Max iter: 100,  Init cf: -0.87829
Current iter: 0  , CF calls: 72, Time elapsed: 0.5s, Current cf val: -9.146
Current iter: 5  , CF calls: 432, Time elapsed: 3.1s, Current cf val: -18.000
Minimum energy found: -17.99999, CF calls: 504, stpt seed: 50905
Time taken: (s): 3.08
Saved result

Begin optimization run. Number of parameters: 24. Stpt seed: 33423, Max iter: 100,  Init